# Integrantes e Link do Vídeo

* Bernardo Maia Coelho - 12542481
* Gustavo Wadas Lopes - 12745640
* Pedro Guilherme dos Reis Teixeira - 12542477
* Pedro Henrique Vilela do Nascimento - 12803492

[Link do vídeo do trabalho 1](https://youtu.be/QR3MZX6I77E)

# Setup

### Libraries

In [1]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from typing import TypedDict
from PIL import Image

### Abre a Janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

### Vertex e Fragment Shaders

In [3]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(model * vec4(position, 1.0));
            out_normal = normals;            
        }
        """

fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        vec3 lightColor = vec3(1.0, 1.0, 1.0);

        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

In [4]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [5]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

# Matriz Model, View, Projection

In [6]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform) # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp)
    
    # print(mat_view)
    # print('\n')
    
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura, inc_fov, inc_near, inc_far
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

# Modelos e Texturas

In [7]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """

    vertices = []
    texture_coords = []
    normals = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"):  # para cada linha do arquivo .obj
        line = line.split("#")[0]
        values = line.strip().split()  # quebra a linha por espaço
        
        if len(values) == 0: continue

        match values[0]:
            case 'v': vertices.append(values[1:4])
            case 'vn': normals.append(values[1:4])
            case 'vt': texture_coords.append(values[1:3])
            case 'usemtl': material = values[1]
            case 'usemat': material = values[1]
            case 'f':
                face = []
                face_texture = []
                face_normals = []

                if len(values[1:]) == 3:  # trigs
                    for v in values[1:]:

                        w = v.split('/')
                        face.append(int(w[0]))
                        face_normals.append(int(w[2]))
                        if len(w) >= 2 and len(w[1]) > 0:
                            face_texture.append(int(w[1]))
                        else:
                            face_texture.append(0)

                    faces.append((face, face_texture, face_normals, material))

                elif len(values[1:]) == 4:  # quads

                    trig1 = [values[1], values[2], values[3]]
                    trig2 = [values[3], values[4], values[1]]

                    for v in trig1:
                        w = v.split('/')
                        face.append(int(w[0]))
                        face_normals.append(int(w[2]))
                        if len(w) >= 2 and len(w[1]) > 0:
                            face_texture.append(int(w[1]))
                        else:
                            face_texture.append(0)

                    for v in trig2:
                        w = v.split('/')
                        face.append(int(w[0]))
                        face_normals.append(int(w[2]))
                        if len(w) >= 2 and len(w[1]) > 0:
                            face_texture.append(int(w[1]))
                        else:
                            face_texture.append(0)

                    faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

In [8]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)


def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura, img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.convert("RGBA").tobytes("raw", "RGBA", 0, -1)

    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height,
                 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

In [9]:
vertices_list = []
normals_list = []
textures_coord_list = []


class Coordinates(TypedDict):
    x = float
    y = float
    z = float


class Model:
    def __init__(self, obj: str, textures: list[int, str], angle: float, r: Coordinates, t: Coordinates, s: Coordinates):
        global vertices_list, textures_coord_list, normals_list

        self.model = load_model_from_file(obj)
        self.textures = textures
        self.angle = angle
        self.r = r
        self.t = t
        self.s = s

        self.vertices = []

        # Se o modelo só tem um vértice, nós utilizamos o primeiro e o último apenas
        if len(self.textures) == 1:
            self.vertices.append(len(vertices_list))

        # Guarda os vértices do modelo
        faces_visited = []
        for face in self.model['faces']:
            if len(self.textures) > 1 and face[3] not in faces_visited:
                self.vertices.append(len(vertices_list))
                faces_visited.append(face[3])

            # Conta a quantidade de vértices
            for vertice_id in face[0]:
                vertices_list.append(self.model['vertices'][vertice_id-1])

            for texture_id in face[1]:
                textures_coord_list.append(self.model['texture'][texture_id-1])

            for normal_id in face[2]:
                normals_list.append(self.model['normals'][normal_id-1])

        self.vertices.append(len(vertices_list))

        for t in textures:
            load_texture_from_file(t[0], t[1])

    def draw(self):
        mat_model = model(self.angle,
                          self.r['x'], self.r['y'], self.r['z'],
                          self.t['x'], self.t['y'], self.t['z'],
                          self.s['x'], self.s['y'], self.s['z'])

        loc_model = glGetUniformLocation(program, "model")
        glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

        # Carrega as texturas para cada face
        for texture in self.textures:
            index = self.textures.index(texture)
            glBindTexture(GL_TEXTURE_2D, texture[0])
            glDrawArrays(GL_TRIANGLES,
                         self.vertices[index], self.vertices[index + 1] - self.vertices[index])

In [10]:
# Guarda os modelos carregados
models = {
    'terreno': Model('terreno/terreno.obj', [(0, 'terreno/terreno.png')], 0.0,
                     {'x': 0.0, 'y': 0.0, 'z': 1.0}, 
                     {'x': 0.0, 'y': -1.01, 'z': 0.0}, 
                     {'x': 200.0, 'y': 200.0, 'z': 200.0}),
    
    'skybox': Model('skybox/skybox.obj', [(1, 'skybox/skybox.png')], 0.0, 
                    {'x': 0.0, 'y': 1.0, 'z': 0.0}, 
                    {'x': 0.0, 'y': 0.0, 'z': 0.0}, 
                    {'x': 3.0, 'y': 3.0, 'z': 3.0}),
    
    'caminho': Model('caminho/caminho.obj', [(2, 'caminho/caminho.jpg')], 0.0,
                     {'x': 0.0, 'y': 1.0, 'z': 0.0},
                     {'x': -54.0, 'y': -1.0, 'z': 0.0},
                     {'x': 95.0, 'y': 1.0, 'z': 15.0}),
    
    'gato': Model('gato/gato.obj', [(3, 'gato/gato.jpg')], 90.0,
                  {'x': -1.0, 'y': 0.0, 'z': 0.0},
                  {'x': 30.0, 'y': 31.3, 'z': -3.7},
                  {'x': 0.05, 'y': 0.05, 'z': 0.05}),
    
    'lobo': Model('lobo/lobo.obj', [(4, 'lobo/lobo.jpg')], -90.0,
                  {'x': 0.0, 'y': 1.0, 'z': 0.0},
                  {'x': 0.0, 'y': -1.0, 'z': 0.0},
                  {'x': 12.0, 'y': 12.0, 'z': 12.0}),
    
    'torre': Model('torre/torre.obj', [(5, 'torre/torre.jpg')], 0.0,
                   {'x': 0.0, 'y': 0.0, 'z': 1.0},
                   {'x': 30.0, 'y': -4.0, 'z': 0.0},
                   {'x': 5.0, 'y': 5.0, 'z': 5.0}),
    
    'arvore': Model('arvore/arvore.obj', [(6, 'arvore/casca.jpg'), (7, 'arvore/folhas.png')], 0.0,
                    {'x': 0.0, 'y': 0.0, 'z': 1.0},
                    {'x': 0.0, 'y': -1.0, 'z': 0.0},
                    {'x': 7.0, 'y': 7.0, 'z': 7.0}),
    
    'golem': Model('golem/golem.obj', [(8, 'golem/golem.tif')], 90.0,
                   {'x': 0.0, 'y': 1.0, 'z': 0.0},
                   {'x': -110.0, 'y': -1.0, 'z': 0.0},
                   {'x': 2.5, 'y': 2.5, 'z': 2.5}),
    
    'gaveta': Model('gaveta/gaveta.obj', [(9, 'gaveta/gaveta.png')], 180.0,
                    {'x': 0.0, 'y': 1.0, 'z': 0.0},
                    {'x': 33.5, 'y': 28.5, 'z': 2.0},
                    {'x': 1.0, 'y': 1.0, 'z': 1.0}),
    
    'caixa': Model('caixa/caixa.obj', [(10, 'caixa/caixa.jpg')], 0.0,
                   {'x': -1.0, 'y': 0.0, 'z': 0.0},
                   {'x': 30.0, 'y': 30.0, 'z': -4.0},
                   {'x': 1.2, 'y': 1.2, 'z': 1.2}),
}

terreno/terreno.png RGBA
skybox/skybox.png RGBA
caminho/caminho.jpg RGB
gato/gato.jpg RGB
lobo/lobo.jpg RGB
torre/torre.jpg RGB
arvore/casca.jpg RGB
arvore/folhas.png RGBA
golem/golem.tif RGB
gaveta/gaveta.png RGBA
caixa/caixa.jpg RGB


# Definição de Desenhos

In [11]:
def desenha_modelos(skybox_inc: float,
                    lobo_x: float, lobo_z: float, lobo_scale: float, lobo_angle: float,
                    arvore_qtd: int, arvore_positions: list[float, float]):

    # Desenha de acordo com as especificações de cada modelo
    for name, model in models.items():
        match(name):
            case 'skybox':
                model.angle = skybox_inc
                model.draw()
                
            case 'lobo':
                model.angle = -90.0 + lobo_angle
                model.t = {'x': lobo_x, 'y': -1.0, 'z': lobo_z}
                model.s = {'x': 12 + lobo_scale, 'y': 12 + lobo_scale, 'z': 12 + lobo_scale}
                model.draw()
            
            case 'arvore':
                for i in range(arvore_qtd):
                    model.t = {'x': arvore_positions[i][0], 'y': -1.0, 'z': arvore_positions[i][1]}
                    model.draw()
            
            case _:
                model.draw()

# Rodando

### Uso da GPU

In [12]:
buffer = glGenBuffers(3)

In [13]:
# Manda os vértices para a GPU
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


# Manda as texturas para a GPU
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

# Manda as normais para a GPU


### Eventos

In [14]:
cameraPos = glm.vec3(25.0,  33.0,  0.0)
cameraFront = glm.vec3(0.0,  0.0, -1.0)
cameraUp = glm.vec3(0.0,  1.0,  0.0)

trap_mouse = True
polygonal_mode = False

inc_fov = 0
inc_near = 0
inc_far = 0
inc_view_up = 0

x_move = 0
z_move = 0
scale = 0
rotate = 0


def check_validity(position):
    cameraAbs = math.sqrt((position.x ** 2) +
                          (position.y ** 2) + (position.z ** 2))
    if (cameraAbs < 138.0) and position.y >= 0:
        return True
    else:
        return False


def key_event(window, key, scancode, action, mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, trap_mouse, inc_fov, inc_near, inc_far, cameraUp, inc_view_up
    global x_move, z_move, scale, rotate

    cameraSpeed = 0.75

    if key == 87 and (action == 1 or action == 2):  # tecla W
        if check_validity(cameraPos + (cameraSpeed * cameraFront)):
            cameraPos += cameraSpeed * cameraFront

    if key == 83 and (action == 1 or action == 2):  # tecla S
        if check_validity(cameraPos - (cameraSpeed * cameraFront)):
            cameraPos -= cameraSpeed * cameraFront

    if key == 65 and (action == 1 or action == 2):  # tecla A
        if check_validity(cameraPos - glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed):
            cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    if key == 68 and (action == 1 or action == 2):  # tecla D
        if check_validity(cameraPos + glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed):
            cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    if key == 80 and action == 1:
        polygonal_mode = not polygonal_mode  # tecla P

    if key == 77 and action == 1:
        trap_mouse = not trap_mouse  # tecla M

    # Move no eixo x e z
    if key == 265:
        x_move -= 0.5  # ArrowUp
    if key == 264:
        x_move += 0.5  # ArrowDown
    if key == 263:
        z_move += 0.25  # ArrowLeft
    if key == 262:
        z_move -= 0.25  # Arrow Right

    # Aumenta e diminui escala
    if key == 90:
        scale += 0.5  # Tecla Z
    if key == 88:
        scale -= 0.5  # Tecla X

    # Roda no sentido horário e anti-horário
    if key == 67:
        rotate -= 3  # Tecla C
    if key == 86:
        rotate += 3  # Tecla V


firstMouse = True
yaw = 20.0
pitch = 5.0
lastX = largura/2
lastY = altura/2


def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset
    pitch += yoffset

    if pitch >= 89.9:
        pitch = 89.9
    if pitch <= -89.9:
        pitch = -89.9

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


glfw.set_key_callback(window, key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Execução Final

In [15]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

In [16]:
glEnable(GL_DEPTH_TEST)

rotacao_inc = 0
while not glfw.window_should_close(window):
    glfw.poll_events()

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    glClearColor(1.0, 1.0, 1.0, 1.0)

    if polygonal_mode == True:
        glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
    else:
        glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)

    if trap_mouse == True:
        glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)
    else:
        glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_NORMAL)

    rotacao_inc += 0.1

    arvores = []
    value = 5
    for i in range(7):
        arvores.append((value - 20*i, 20))
        arvores.append((value - 20*i, -20))
        arvores.append((value - 20*i, 40))
        arvores.append((value - 20*i, -40))
        arvores.append((value - 20*i, 60))
        arvores.append((value - 20*i, -60))

    desenha_modelos(rotacao_inc, x_move, z_move, scale,
                    rotate, len(arvores), arvores)

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    print(mat_view)
    
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)

    glfw.swap_buffers(window)

glfw.terminate()

[[ -0.39073128   0.           0.9205048    9.768282  ]
 [  0.50268877   0.83771926   0.2133788  -40.211952  ]
 [ -0.7711246    0.54610115  -0.3273231    1.2567768 ]
 [  0.           0.           0.           1.        ]]
[[ -0.41469288   0.           0.90996146  10.367322  ]
 [  0.6741026    0.6717219    0.30720592 -39.019386  ]
 [ -0.611241     0.7408035   -0.27855825  -9.165489  ]
 [  0.           0.           0.           1.        ]]
[[-7.6614451e-01  0.0000000e+00  6.4266837e-01  1.9153612e+01]
 [ 6.4266735e-01  1.7450980e-03  7.6614332e-01 -1.6124273e+01]
 [-1.1215192e-03  9.9999845e-01 -1.3369971e-03 -3.2971912e+01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[[-8.5464728e-01  0.0000000e+00  5.1920909e-01  2.1366182e+01]
 [ 5.1920831e-01  1.7449411e-03  8.5464597e-01 -1.3037790e+01]
 [-9.0598920e-04  9.9999845e-01 -1.4913091e-03 -3.2977303e+01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[[-9.3480474e-01  0.0000000e+00  3.5516202e-01  2.33